In [1]:
#ServiceBusClient is a Class (attributes/properties and methods/functions)
#from azure.servicebus import ServiceBusClient, ServiceBusMessage
from azure.servicebus import ServiceBusClient as QueueClient
from azure.servicebus import ServiceBusMessage as Message

In [2]:
connstr = "Endpoint=sb://trainingeventhubfirst.servicebus.windows.net/;SharedAccessKeyName=SendMessagestofirsteventhubqueue;SharedAccessKey=vzLrFXyQogYzgFIdRnl220qZGEHBatVxDdVF9nAwOGo="
queue_client = QueueClient.from_connection_string(connstr)
queue_client
sender = queue_client.get_queue_sender(queue_name="fristevenethubqueue")
print(queue_client)
print(sender)

In [ ]:
for i in range(5):
    name =  str(i) + "Maaz"
    msg = Message(name)
    sender.send_messages(msg)
    print('msg ', name)

# Receive Event Messages

In [ ]:
pip install azure.eventhub

In [3]:
from azure.eventhub import EventHubConsumerClient

In [4]:
connstr = "Endpoint=sb://trainingeventhubfirst.servicebus.windows.net/;SharedAccessKeyName=OnlyReceiveMessagePolicy;SharedAccessKey=WV6mpgCPzDhdwFbEgrT6HGc4DdEUckPTo52wLf8vyOU="
eventhubname = "fristevenethubqueue"
consumer_groupdef = "$Default"
consumer_group1 = 'newconsumergroup'
consumer_group2 = 'secondconsumergroup'

In [5]:
def on_event(partition_context, event):
    print(event.body_as_str(encoding='UTF-8'))
        #print("Received event from partition {}".format(partition_context.partition_id))

def on_partition_initialize(partition_context):
    # Put your code here.
    print("Partition: {} has been initialized.".format(partition_context.partition_id))


def on_partition_close(partition_context, reason):
    # Put your code here.
    print("Partition: {} has been closed, reason for closing: {}.".format(partition_context.partition_id,reason))


def on_error(partition_context, error):
    # Put your code here. partition_context can be None in the on_error callback.
    if partition_context:
        print("An exception: {} occurred during receiving from Partition: {}.".format(partition_context.partition_id,error))
    else:
        print("An exception: {} occurred during the load balance process.".format(error))


In [6]:
client1 = EventHubConsumerClient.from_connection_string(conn_str=connstr, consumer_group = consumer_group1, eventhub_name=eventhubname, idle_timeout = 30, auth_timeout = 10)
client1

In [9]:
try:
    with client1:
        ## "-1" is from the beginning of the partition. OR starting_position=starting_position
        ##, track_last_enqueued_event_properties=False, starting_position="@latest", partition_id = "0", 
        client1.receive(on_event=on_event,on_partition_initialize=on_partition_initialize,on_partition_close=on_partition_close,on_error=on_error,starting_position="-1", timeout=10) 
except KeyboardInterrupt:
    print('Stopped receiving.')

Partition: 3 has been initialized.
Partition: 4 has been initialized.
Partition: 2 has been initialized.
Partition: 1 has been initialized.
Partition: 0 has been initialized.
1Maaz
{
"fname" : "rajan",
"lname" : "gopal",
"city" : "abu dhabi"

}
{
"fname" : "rajan",
"lname" : "gopal",
"city" : "sharjah"

}
6Maaz
0Maaz
3Maaz
4Maaz
11Maaz
5Maaz
8Maaz
9Maaz
16Maaz
10Maaz
13Maaz
14Maaz
21Maaz
15Maaz
18Maaz
19Maaz
{
"fname" : "ramesh",
"lname" : "gopal",
"city" : "dubai"

}
26Maaz
20Maaz
23Maaz
24Maaz
2Maaz
31Maaz
25Maaz
28Maaz
29Maaz
7Maaz
36Maaz
30Maaz
33Maaz
34Maaz
12Maaz
41Maaz
35Maaz
38Maaz
39Maaz
17Maaz
46Maaz
40Maaz
43Maaz
44Maaz
22Maaz
1Maaz
45Maaz
48Maaz
49Maaz
27Maaz
1Maaz
0Maaz
3Maaz
4Maaz
32Maaz
1Maaz
0Maaz
3Maaz
4Maaz
37Maaz
1Maaz
0Maaz
3Maaz
4Maaz
42Maaz
1Maaz
0Maaz
3Maaz
4Maaz
47Maaz
1Maaz
0Maaz
3Maaz
4Maaz
2Maaz
3Maaz
0Maaz
3Maaz
4Maaz
2Maaz
3Maaz
2Maaz
0Maaz
1Maaz
2Maaz
2Maaz
0Maaz
1Maaz
2Maaz
2Maaz
2Maaz
4Maaz
4Maaz
990Maaz
991Maaz
992Maaz
993Maaz
994Maaz
99   0Maaz
99   1M

In [ ]:
import threading, time

In [ ]:
receive_duration = 5
thread = threading.Thread(
    target=client1.receive,
    kwargs={
        "on_event": on_event,
        "on_partition_initialize": on_partition_initialize,
        "on_partition_close": on_partition_close,
        "on_error": on_error,
        "starting_position": "-1",  # "-1" is from the beginning of the partition.
    }
)

thread.daemon = True
thread.start()
time.sleep(receive_duration)
#client1.close()
thread.join()

In [ ]:
partition_0_prop = client.get_partition_properties("0")
partition_0_last_enqueued_sequence_number = partition_0_prop["last_enqueued_sequence_number"]
# client will receive messages from the position of the third from last event on partition 0.
starting_position = {"0": partition_0_last_enqueued_sequence_number - 3,}